In [1]:
# --- Imports ---
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from azure.monitor.opentelemetry.exporter import AzureMonitorTraceExporter # Uniquement le TraceExporter
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

print("Bibliothèques importées.")

# Optionnel: Journalisation interne OpenTelemetry pour le débogage
# import logging
# logging.basicConfig(level=logging.DEBUG)
# logging.getLogger("opentelemetry.sdk").setLevel(logging.DEBUG)
# logging.getLogger("azure.monitor.opentelemetry.exporter").setLevel(logging.DEBUG)


# --- Configuration Application Insights (Traces uniquement) ---
load_dotenv()
connection_string = os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING")
print(f"DEBUG: Connection String lue depuis .env: '{connection_string}'")


tracer = None
span_processor = None

if connection_string:
    try:
        exporter = AzureMonitorTraceExporter.from_connection_string(connection_string)
        trace_provider = TracerProvider() # Important: créer un nouveau provider
        trace.set_tracer_provider(trace_provider) # Le définir globalement
        tracer = trace.get_tracer("NotebookFeedbackTracer") # Nom du tracer
        span_processor = BatchSpanProcessor(exporter)
        trace_provider.add_span_processor(span_processor) # L'ajouter au provider spécifique

        print("✅ Configuration Traces Application Insights réussie.")

        # Envoyer un span de test au démarrage
        with tracer.start_as_current_span("NotebookTool_StartupTestSpan") as span:
            span.set_attribute("status", "trace_initialized")
            span.set_attribute("test_span_value", 789)
        print("   Span de démarrage de test envoyé.")
        if span_processor:
            span_processor.force_flush(timeout_millis=15000)
            print("   Flush des spans de test tenté.")

    except Exception as e:
        print(f"❌ ERREUR lors de la configuration d'Application Insights (Traces): {e}")
        import traceback
        traceback.print_exc()
        tracer = None
else:
    print("⚠️ ATTENTION: Chaîne de connexion Application Insights non trouvée dans .env.")
    tracer = None

# --- Configuration API Déployée ---
API_URL = "https://airparadis-sentiment-api.azurewebsites.net/predict"
print(f"URL de l'API configurée : {API_URL}")

Bibliothèques importées.
DEBUG: Connection String lue depuis .env: 'InstrumentationKey=36206470-0647-4efb-b0b2-808d99bd9cee;IngestionEndpoint=https://francecentral-1.in.applicationinsights.azure.com/;LiveEndpoint=https://francecentral.livediagnostics.monitor.azure.com/;ApplicationId=02ecc65c-862b-41f8-a56e-24c464fd9690'
✅ Configuration Traces Application Insights réussie.
   Span de démarrage de test envoyé.
   Flush des spans de test tenté.
URL de l'API configurée : https://airparadis-sentiment-api.azurewebsites.net/predict


In [ ]:
# --- Création des Widgets pour l'Interface ---

# 1. Zone de texte pour saisir le tweet
tweet_input = widgets.Textarea(
    value='',
    placeholder='Entrez votre tweet ici...',
    description='Tweet:',
    layout={'width': '90%', 'height': '80px'}, # Ajuster la taille si besoin
    disabled=False
)

# 2. Bouton pour lancer la prédiction
predict_button = widgets.Button(
    description="Analyser Sentiment",
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Cliquez pour obtenir la prédiction de sentiment depuis l\'API',
    icon='search' # Optionnel: ajoute une petite icône
)

# 3. Zone pour afficher les résultats et les boutons de feedback
output_area = widgets.Output(layout={'border': '1px solid black', 'padding': '10px', 'width': '90%'})

# 4. Boutons de feedback (initialement cachés, gérés dans l'output_area)
yes_button = widgets.Button(description="Oui, correct", button_style='success', icon='check')
no_button = widgets.Button(description="Non, incorrect", button_style='danger', icon='times')

# 5. Afficher la zone de saisie et le bouton principal
print("Affichage des widgets de saisie...")
display(tweet_input)
display(predict_button)
display(output_area) # Afficher la zone où les résultats apparaîtront

print("Widgets créés et affichés. Prêt à définir les actions.")

Affichage des widgets de saisie...


Textarea(value='', description='Tweet:', layout=Layout(height='80px', width='90%'), placeholder='Entrez votre …

Button(button_style='info', description='Analyser Sentiment', icon='search', style=ButtonStyle(), tooltip="Cli…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Widgets créés et affichés. Prêt à définir les actions.


In [5]:
# --- Définition des Actions des Boutons ---

# Variable globale pour stocker le dernier tweet et la dernière prédiction
# Cela nous permet de les réutiliser lorsque l'utilisateur donne son feedback
last_processed_tweet = None
last_prediction_data = None

# Fonction appelée lorsque l'utilisateur clique sur "Non, incorrect"
def on_no_button_clicked(b):
    global last_processed_tweet, last_prediction_data, tracer, span_processor # span_processor pour le flush

    with output_area:
        clear_output(wait=True)
        print("Merci pour votre feedback !")
        print(f"Tweet : {last_processed_tweet}")
        print(f"Prédiction (incorrecte) : {last_prediction_data}")

        if tracer and span_processor and last_processed_tweet and last_prediction_data:
            try:
                # Envoyer un span à Application Insights
                with tracer.start_as_current_span("IncorrectPredictionFeedbackSpan") as span: # Nom du span
                    # Les attributs seront les customDimensions dans la table dependencies/traces
                    span.set_attribute("feedback.tweet_text", last_processed_tweet)
                    span.set_attribute("feedback.predicted_sentiment", last_prediction_data.get('sentiment_label', 'N/A'))
                    span.set_attribute("feedback.predicted_score", str(last_prediction_data.get('sentiment_score', 'N/A')))
                    span.set_attribute("feedback.type", "UserReportedError") # Catégorisation
                
                print("   Span 'IncorrectPredictionFeedbackSpan' envoyé.")
                
                print("   Tentative de forcer le flush du span vers Azure...")
                timeout_seconds = 15
                success = span_processor.force_flush(timeout_millis=timeout_seconds * 1000)
                if success:
                    print(f"   ✅ Flush du span réussi dans les {timeout_seconds}s.")
                else:
                    print(f"   ⚠️ ATTENTION: Flush du span a dépassé le timeout de {timeout_seconds}s.")

            except Exception as e:
                print(f"❌ ERREUR lors de l'envoi du span de feedback à App Insights: {e}")
                import traceback
                traceback.print_exc()
        elif not tracer:
            print("⚠️ Tracer Application Insights désactivé, feedback non envoyé.")
        else:
            print("⚠️ Données de tweet/prédiction/processeur manquantes, feedback non envoyé.")


# Fonction appelée lorsque l'utilisateur clique sur "Oui, correct"
def on_yes_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        print("Super ! Merci pour la confirmation.")
        # Optionnel: Envoyer un événement "CorrectPrediction" si on veut aussi tracker les succès
        # if tracer and last_processed_tweet and last_prediction_data:
        #     with tracer.start_as_current_span("CorrectPredictionFeedback") as span:
        #         span.set_attribute("tweet_text", last_processed_tweet)
        #         # ... autres attributs
        #     print("Confirmation de prédiction correcte envoyée (optionnel).")


# Fonction principale appelée lorsque l'utilisateur clique sur "Analyser Sentiment"
def on_predict_button_clicked(b):
    global last_processed_tweet, last_prediction_data # Indiquer qu'on modifie les variables globales

    tweet_text = tweet_input.value
    last_processed_tweet = tweet_text # Sauvegarder le tweet actuel

    if not tweet_text.strip():
        with output_area:
            clear_output(wait=True)
            print("⚠️ Veuillez entrer un tweet avant d'analyser.")
        return

    with output_area: # Toutes les sorties (print, display) iront dans cette zone
        clear_output(wait=True) # Effacer les anciens résultats
        print(f"Analyse du tweet en cours : \"{tweet_text[:100]}...\"") # Afficher un extrait

        try:
            # Préparer le payload JSON pour l'API
            payload = {"tweet_text": tweet_text}
            headers = {"Content-Type": "application/json"}

            # Appeler l'API
            response = requests.post(API_URL, data=json.dumps(payload), headers=headers, timeout=30) # Timeout de 30s
            response.raise_for_status()  # Lève une exception pour les codes d'erreur HTTP (4xx, 5xx)

            # Récupérer la réponse JSON
            prediction = response.json()
            last_prediction_data = prediction # Sauvegarder la prédiction actuelle

            print("\n--- Prédiction Reçue ---")
            # Afficher de manière un peu plus structurée si possible
            # Si pandas est disponible, on peut faire un joli affichage
            try:
                df_prediction = pd.DataFrame([prediction])
                display(df_prediction)
            except NameError: # Si pandas n'est pas importé
                print(f"Sentiment : {prediction.get('sentiment_label', 'N/A')}")
                print(f"Score     : {prediction.get('sentiment_score', 'N/A'):.4f}")


            print("\nLa prédiction était-elle correcte ?")
            # Afficher les boutons de feedback
            # Recréer les boutons à chaque fois pour réassocier les callbacks (plus simple dans ce contexte)
            yes_btn = widgets.Button(description="Oui, correct", button_style='success', icon='check')
            no_btn = widgets.Button(description="Non, incorrect", button_style='danger', icon='times')

            yes_btn.on_click(on_yes_button_clicked)
            no_btn.on_click(on_no_button_clicked)

            # Afficher les boutons côte à côte
            buttons_hbox = widgets.HBox([yes_btn, no_btn])
            display(buttons_hbox)

        except requests.exceptions.HTTPError as http_err:
            print(f"❌ Erreur HTTP de l'API : {http_err}")
            try:
                # Essayer d'afficher le détail de l'erreur de l'API si disponible
                error_detail = response.json().get("detail", "Aucun détail fourni.")
                print(f"   Détail API : {error_detail}")
            except:
                print(f"   Réponse brute : {response.text[:500]}") # Limiter la taille
        except requests.exceptions.ConnectionError as conn_err:
            print(f"❌ Erreur de connexion à l'API : {conn_err}")
            print(f"   Vérifiez que l'API est bien démarrée et accessible à {API_URL}")
        except requests.exceptions.Timeout as timeout_err:
            print(f"❌ Timeout lors de l'appel à l'API : {timeout_err}")
        except json.JSONDecodeError:
            print("❌ Erreur : L'API n'a pas retourné une réponse JSON valide.")
            print(f"   Réponse brute : {response.text[:500]}")
        except Exception as e:
            print(f"❌ Une erreur inattendue est survenue : {e}")
            import traceback
            traceback.print_exc() # Affiche la trace complète pour le débogage


# Lier la fonction au clic du bouton de prédiction
predict_button.on_click(on_predict_button_clicked)

print("Fonctions de callback définies et liées aux boutons.")

Fonctions de callback définies et liées aux boutons.
